In [1]:
import pandas as pd
import numpy as np
import re
from scipy import sparse
import scipy.io as sio

# Input

In [2]:
D1 = '../../experiments/exp10x/raw/vdj_v1_hs_aggregated_donor1_binarized_matrix.csv'
D2 = '../../experiments/exp10x/raw/vdj_v1_hs_aggregated_donor2_binarized_matrix.csv'
D3 = '../../experiments/exp10x/raw/vdj_v1_hs_aggregated_donor3_binarized_matrix.csv'
D4 = '../../experiments/exp10x/raw/vdj_v1_hs_aggregated_donor4_binarized_matrix.csv'

# Output

In [3]:
DIR = "../../experiments/exp10x/run1/tcr/cellranger_tot/outs/multi/count/raw_feature_bc_matrix/"

In [4]:
MTX = DIR + "matrix"
GEM = DIR + "barcodes.tsv.gz"
BRC = DIR + "features.tsv.gz"

# Load

In [5]:
d1 = pd.read_csv(D1)
d2 = pd.read_csv(D2)
d3 = pd.read_csv(D3)
d4 = pd.read_csv(D4)

In [6]:
d1.columns.to_list()

['barcode',
 'donor',
 'cell_clono_cdr3_aa',
 'cell_clono_cdr3_nt',
 'CD3',
 'CD19',
 'CD45RA',
 'CD4',
 'CD8a',
 'CD14',
 'CD45RO',
 'CD279_PD-1',
 'IgG1',
 'IgG2a',
 'IgG2b',
 'CD127',
 'CD197_CCR7',
 'HLA-DR',
 'A0101_VTEHDTLLY_IE-1_CMV',
 'A0201_KTWGQYWQV_gp100_Cancer',
 'A0201_ELAGIGILTV_MART-1_Cancer',
 'A0201_CLLWSFQTSA_Tyrosinase_Cancer',
 'A0201_IMDQVPFSV_gp100_Cancer',
 'A0201_SLLMWITQV_NY-ESO-1_Cancer',
 'A0201_KVAELVHFL_MAGE-A3_Cancer',
 'A0201_KVLEYVIKV_MAGE-A1_Cancer',
 'A0201_CLLGTYTQDV_Kanamycin-B-dioxygenase',
 'A0201_LLDFVRFMGV_EBNA-3B_EBV',
 'A0201_LLMGTLGIVC_HPV-16E7_82-91',
 'A0201_CLGGLLTMV_LMP-2A_EBV',
 'A0201_YLLEMLWRL_LMP1_EBV',
 'A0201_FLYALALLL_LMP2A_EBV',
 'A0201_GILGFVFTL_Flu-MP_Influenza',
 'A0201_GLCTLVAML_BMLF1_EBV',
 'A0201_NLVPMVATV_pp65_CMV',
 'A0201_ILKEPVHGV_RT_HIV',
 'A0201_FLASKIGRLV_Ca2-indepen-Plip-A2',
 'A2402_CYTWNQMNL_WT1-(235-243)236M_Y',
 'A0201_RTLNAWVKV_Gag-protein_HIV',
 'A0201_KLQCVDLHV_PSA146-154',
 'A0201_LLFGYPVYV_HTLV-1',
 'A0201_SL

In [7]:
regex_hla = re.compile('([ABC]\d{4})')
regex_pep = re.compile('_([A-Z]{5,})_')

dfs = list()
for df in [d1,d2,d3,d4]:
    cols = ['barcode']
    newc = ['barcode']
    for column in df.columns:
        if not column.endswith('binder'):
            match_hla = regex_hla.search(column)
            match_pep = regex_pep.search(column)
            if (match_hla is not None) & (match_pep is not None):
                cols.append(column)
                newc.append(match_pep.group(1) + '_' + match_hla.group(1))

    df = df[cols]
    df.columns = newc
    
    dfs.append(df)

In [8]:
df = pd.concat(dfs, keys=['s1','s2','s3','s4'])

In [9]:
df

barcode  VTEHDTLLY_A0101  KTWGQYWQV_A0201  \
s1 0       AAACCTGAGACAAAGG-4              0.0              0.0   
   1      AAACCTGAGACTGTAA-34              0.0              0.0   
   2       AAACCTGAGAGCCCAA-5              0.0              0.0   
   3      AAACCTGAGAGCTGCA-24              0.0              0.0   
   4       AAACCTGAGAGGGATA-8              0.0              0.0   
...                       ...              ...              ...   
s4 27303   TTTGTCATCGTTTGCC-4              0.0              0.0   
   27304   TTTGTCATCTACCAGA-1              0.0              0.0   
   27305   TTTGTCATCTCTAAGG-8              0.0              0.0   
   27306   TTTGTCATCTGCAAGT-4              0.0              0.0   
   27307   TTTGTCATCTTAGAGC-2              0.0              0.0   

          ELAGIGILTV_A0201  CLLWSFQTSA_A0201  IMDQVPFSV_A0201  \
s1 0                   0.0               0.0              0.0   
   1                   0.0               0.0              0.0   
   2                   0.0               0.0              0.0   
   3                   0.0               0.0              0.0   
   4                   0.0               0.0              0.0   
...                    ...               ...              ...   
s4 27303               0.0               0.0              0.0   
   27304               0.0               0.0              0.0   
   27305               0.0               0.0              0.0   
   27306               0.0               0.0              0.0   
   27307               0.0               0.0              0.0   

          SLLMWITQV_A0201  KVAELVHFL_A0201  KVLEYVIKV_A0201  CLLGTYTQDV_A0201  \
s1 0                  0.0              0.0              0.0               0.0   
   1                  0.0              0.0              0.0               0.0   
   2                  0.0              0.0              0.0               1.0   
   3                  0.0              0.0              0.0               0.0   
   4                  0.0              1.0              0.0               0.0   
...                   ...              ...              ...               ...   
s4 27303              0.0              0.0              0.0               0.0   
   27304              0.0              0.0              1.0               0.0   
   27305              0.0              0.0              0.0               0.0   
   27306              0.0              0.0              0.0               0.0   
   27307              0.0              0.0              0.0               0.0   

          ...  RPHERNGFTVL_B0702  RAKFKQLL_B0801  ELRRKMMYM_B0801  \
s1 0      ...                0.0             0.0              0.0   
   1      ...                0.0             0.0              0.0   
   2      ...                0.0             0.0              0.0   
   3      ...                0.0             1.0              0.0   
   4      ...                0.0             0.0              0.0   
...       ...                ...             ...              ...   
s4 27303  ...                0.0             0.0              0.0   
   27304  ...                0.0             0.0              0.0   
   27305  ...                0.0             0.0              0.0   
   27306  ...                0.0             0.0              0.0   
   27307  ...                0.0             0.0              0.0   

          FLRGRAYGL_B0801  SLEGGGLGY_A0101  STEGGGLAY_A0101  ALIAPVHAV_A0201  \
s1 0                  0.0              0.0              0.0              0.0   
   1                  0.0              0.0              0.0              0.0   
   2                  0.0              0.0              0.0              0.0   
   3                  1.0              0.0              0.0              0.0   
   4                  0.0              0.0              0.0              0.0   
...                   ...              ...              ...              ...   
s4 27303              0.0              0.0              0.0              0

In [10]:
df.reset_index(inplace=True)
df

,level_0,level_1,barcode,VTEHDTLLY_A0101,KTWGQYWQV_A0201,ELAGIGILTV_A0201,CLLWSFQTSA_A0201,IMDQVPFSV_A0201,SLLMWITQV_A0201,KVAELVHFL_A0201,...,RPHERNGFTVL_B0702,RAKFKQLL_B0801,ELRRKMMYM_B0801,FLRGRAYGL_B0801,SLEGGGLGY_A0101,STEGGGLAY_A0101,ALIAPVHAV_A0201,AYSSAGASI_A2402,GPAESAAGL_B0702,AAKGRGAAL_B0801
0,s1,0,AAACCTGAGACAAAGG-4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,s1,1,AAACCTGAGACTGTAA-34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,s1,2,AAACCTGAGAGCCCAA-5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,s1,3,AAACCTGAGAGCTGCA-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,s1,4,AAACCTGAGAGGGATA-8,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189507,s4,27303,TTTGTCATCGTTTGCC-4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189508,s4,27304,TTTGTCATCTACCAGA-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189509,s4,27305,TTTGTCATCTCTAAGG-8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189510,s4,27306,TTTGTCATCTGCAAGT-4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
df.barcode = df.barcode + '-' + df.level_0
df

,level_0,level_1,barcode,VTEHDTLLY_A0101,KTWGQYWQV_A0201,ELAGIGILTV_A0201,CLLWSFQTSA_A0201,IMDQVPFSV_A0201,SLLMWITQV_A0201,KVAELVHFL_A0201,...,RPHERNGFTVL_B0702,RAKFKQLL_B0801,ELRRKMMYM_B0801,FLRGRAYGL_B0801,SLEGGGLGY_A0101,STEGGGLAY_A0101,ALIAPVHAV_A0201,AYSSAGASI_A2402,GPAESAAGL_B0702,AAKGRGAAL_B0801
0,s1,0,AAACCTGAGACAAAGG-4-s1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,s1,1,AAACCTGAGACTGTAA-34-s1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,s1,2,AAACCTGAGAGCCCAA-5-s1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,s1,3,AAACCTGAGAGCTGCA-24-s1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,s1,4,AAACCTGAGAGGGATA-8-s1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189507,s4,27303,TTTGTCATCGTTTGCC-4-s4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189508,s4,27304,TTTGTCATCTACCAGA-1-s4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189509,s4,27305,TTTGTCATCTCTAAGG-8-s4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189510,s4,27306,TTTGTCATCTGCAAGT-4-s4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
pd.Series(cols[1:]).str.split('_',expand=True).sort_values(by=[0,3,1])#.to_excel('../tmp_files/barcode_specificity.xlsx')

,0,1,2,3
0,A0101,VTEHDTLLY,IE-1,CMV
44,A0101,SLEGGGLGY,NC,None
45,A0101,STEGGGLAY,NC,None
10,A0201,LLMGTLGIVC,HPV-16E7,82-91
16,A0201,NLVPMVATV,pp65,CMV
3,A0201,CLLWSFQTSA,Tyrosinase,Cancer
2,A0201,ELAGIGILTV,MART-1,Cancer
4,A0201,IMDQVPFSV,gp100,Cancer
1,A0201,KTWGQYWQV,gp100,Cancer
6,A0201,KVAELVHFL,MAGE-A3,Cancer


In [12]:
df.set_index('barcode', inplace=True)

In [13]:
df.drop(columns=['level_0','level_1'], inplace=True)

In [15]:
sparse_matrix = sparse.csr_matrix(df.T.values)

In [16]:
sio.mmwrite(MTX, sparse_matrix)

In [78]:
features = pd.DataFrame(df.columns)[0].str.split('_', expand=True)
features.columns = ['pep','hla']

In [79]:
features['capture'] = 'Antibody Capture'

In [85]:
features[['pep','pep','capture']].to_csv(BRC, header=None, index=False, sep='\t')

In [17]:
pd.DataFrame(df.index).to_csv(GEM, header=None, index=False, sep='\t')

In [87]:
features

,pep,hla,identifier,capture
0,VTEHDTLLY,A0101,VTEHDTLLY,Antibody Capture
1,KTWGQYWQV,A0201,KTWGQYWQV,Antibody Capture
2,ELAGIGILTV,A0201,ELAGIGILTV,Antibody Capture
3,CLLWSFQTSA,A0201,CLLWSFQTSA,Antibody Capture
4,IMDQVPFSV,A0201,IMDQVPFSV,Antibody Capture
5,SLLMWITQV,A0201,SLLMWITQV,Antibody Capture
6,KVAELVHFL,A0201,KVAELVHFL,Antibody Capture
7,KVLEYVIKV,A0201,KVLEYVIKV,Antibody Capture
8,CLLGTYTQDV,A0201,CLLGTYTQDV,Antibody Capture
9,LLDFVRFMGV,A0201,LLDFVRFMGV,Antibody Capture
